# アクティブユーザー数 

本資料は、アサインナビにおけるアクティブユーザー数を算出するためのものである。
 - アクティブユーザー数とは
  - 過去30日以内にログイン or 過去90日以内にアクション の法人数
  - 過去90日以内にログイン or 過去120日以内にアクション の個人数

※日数は暫定  
※アクションとは、案件登録、レジュメ登録、スカウト、提案、イベント参加

### import

In [85]:
import datetime
import codecs
import pandas as pd
from pandas import DataFrame

### 基準日設定

ここで、基準となる日を設定することができる。  
現時点では過去30日以内、過去90日以内、過去120日以内としている。

In [51]:
base_date = datetime.datetime(2016, 11, 1, 0, 0, 0)
base_date_str = base_date.strftime("%Y-%m-%d %H:%M:%S")
before_30days = base_date + datetime.timedelta(days=-30)
before_30days_str = before_30days.strftime("%Y-%m-%d %H:%M:%S")
before_90days = base_date + datetime.timedelta(days=-90)
before_90days_str = before_90days.strftime("%Y-%m-%d %H:%M:%S")
before_120days = base_date + datetime.timedelta(days=-120)
before_120days_str = before_120days.strftime("%Y-%m-%d %H:%M:%S")

### 案件登録

In [83]:
with codecs.open('./csv/opportunities.csv', 'r', 'Shift-JIS', 'ignore') as file:
    df_opp_temp = pd.read_csv(file, index_col='id').fillna('')
df_opportunities = df_opp_temp[(df_opp_temp['company'] != '') & (df_opp_temp['published'] != '') & (df_opp_temp['draft'] != '1') & (df_opp_temp['deleted'] != '1')].ix[:,['id','published','company','isPaying']]
df_opp_group = DataFrame(df_opportunities[['company','published']].groupby('company')['published'].max())
# df_opp_group[df_opp_group.published > before_90days_str]

### レジュメ登録

In [57]:
with codecs.open('./csv/resumes.csv', 'r', 'Shift-JIS', 'ignore') as file:
    df_res_temp = pd.read_csv(file, index_col='id').fillna('')
df_resumes = df_res_temp[(df_res_temp['company'] != '') & (df_res_temp['draft'] != '1') & (df_res_temp['deleted'] != '1')].ix[:,['id','registed','modified','company','isPaying']]
df_res_group = DataFrame(df_resumes[['company','modified']].groupby('company')['modified'].max())
# df_res_group[df_res_group.modified > before_90days_str]

### スカウト

In [77]:
with codecs.open('./csv/scouts.csv', 'r', 'Shift-JIS', 'ignore') as file:
    df_scouts_temp = pd.read_csv(file, index_col='message_id').fillna('')
df_scouts = df_scouts_temp[(df_scouts_temp['company'] != '')].ix[:,['message_id','scout_status','created','company','isPaying']]
df_scouts.rename(columns={'message_id': 'id'}, inplace=True)
df_scouts_group = DataFrame(df_scouts[['company','created']].groupby('company')['created'].max())
# df_scouts_group[df_scouts_group.created > before_90days_str]

### 提案

In [67]:
with codecs.open('./csv/proposed_resumes.csv', 'r', 'Shift-JIS', 'ignore') as file:
    df_pro_temp = pd.read_csv(file, index_col='id').fillna('')
df_proposed_resumes = df_pro_temp[(df_pro_temp['company'] != '')].ix[:,['id','created','modified','company','isPaying']]
df_pro_group = DataFrame(df_proposed_resumes[['company','modified']].groupby('company')['modified'].max())
# df_pro_group[df_pro_group.modified > before_90days_str]

### イベント

In [78]:
with codecs.open('./csv/events.csv', 'r', 'Shift-JIS', 'ignore') as file:
    df_eve_temp = pd.read_csv(file).fillna('')
df_events = df_eve_temp[(df_eve_temp['company'] != '')].ix[:,['id','date','company','isPaying']]
df_eve_group = DataFrame(df_events[['company','date']].groupby('company')['date'].max())
df_eve_group[datetime.datetime.strptime(df_eve_group.date, "%Y/%m/%d")  > before_90days_str]
# df_eve_group[df_eve_group.date > before_90days_str]

TypeError: strptime() argument 1 must be string, not Series

### 会社テーブル

In [59]:
with codecs.open('./csv/companies.csv', 'r', 'Shift-JIS', 'ignore') as file:
    df_companies = pd.read_csv(file, index_col='id', usecols=['id','uuid','name','url','created','updated','last_login','isPaying']).fillna('')
df_companies[:20]

,uuid,name,url,created,updated,last_login,isPaying
id,,,,,,,
1,988066285892,株式会社 エル・ティー・エス,http://www.lt-s.jp/,2014-10-23 21:11:29,2016-06-20 17:45:34,2016-10-21 11:24:02,スタンダードプラン
2,247113551374,アクロスロード株式会社,http://www.axrossroad.co.jp/,2014-10-24 10:19:18,2016-05-08 23:33:45,2015-04-09 11:39:28,ライトプラン
3,787121381699,有限会社アイ・ティ・ソリューション,http://itsol.jp/,2014-10-24 12:29:49,2015-01-14 19:21:45,2015-01-14 14:47:34,ライトプラン
4,777129265687,ザックシステム株式会社,http://www.zaq.co.jp/,2014-10-24 14:41:01,2016-10-26 23:34:36,2016-06-15 17:44:34,ライトプラン
5,364131262019,株式会社シー・エス・エス,http://www.css-net.co.jp/,2014-10-24 15:14:29,2016-03-15 09:51:27,2016-03-10 17:52:00,ライトプラン
6,754135652071,ジャパンソフト,http://www.jpn-soft.co.jp,2014-10-24 16:27:38,2016-05-08 23:34:13,2015-03-13 16:03:51,ライトプラン
7,905142366060,株式会社LDS,http://l-ds.jp/,2014-10-24 18:19:22,2016-10-26 23:34:37,2016-10-09 09:10:46,ライトプラン
8,495155891538,株式会社バーネット,http://www.barnet.co.jp,2014-10-24 22:04:54,2016-10-26 23:34:24,2016-10-17 12:23:13,スタンダードプラン
9,656218181448,株式会社ギークフィード,http://www.geekfeed.co.jp,2014-10-25 15:23:11,2016-10-26 23:34:00,2016-01-22 09:22:06,ライトプラン
